In [1]:
!pip uninstall huggingface_hub -y && pip install git+https://github.com/huggingface/huggingface_hub
!pip install jiwer

Found existing installation: huggingface-hub 0.10.1
Uninstalling huggingface-hub-0.10.1:
  Successfully uninstalled huggingface-hub-0.10.1
  Cloning https://github.com/huggingface/huggingface_hub to /tmp/pip-req-build-qzj3ut0b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/huggingface_hub /tmp/pip-req-build-qzj3ut0b
  Resolved https://github.com/huggingface/huggingface_hub to commit 79c6320ed0345f9fed0a92a5fb40f62334074ac8
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for huggingface-hub: filename=huggingface_hub-0.13.0.dev0-py3-none-any.whl size=189157 sha256=3187391c7d62b517216f48020ff645b62889ac5e80c7add1702f7bc368640496
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy56936r/wheels/74/e6/c6/b1af4c1e6861c2ac8b8d73c6d28603215917f9b268787b59f8
Successfully built huggingface-hub
ERROR: pip's dependency resolver do

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )


In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = '/kaggle/input/t5-asr-corrector-raw-first-attempt/t5_asr_bn_corrector_first_attempt'
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_name, local_files_only=True).to('cuda')

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [5]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [6]:
df = pd.read_parquet("/kaggle/input/benchmark-1/benchmark.parquet")

In [7]:
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,/kaggle/input/denoiser-for-benchmark/denoised/...,056711c865,openslr,denoised,রাজনৈতিক সমাবেশ,3.168,-37.201962,রাজনৈতিক সমাপেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,...,"['C', 'S']",50,"['C', 'C']",0,"['C', 'C']",0,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ
49148,/kaggle/input/denoiser-for-benchmark/denoised/...,a3230c3684,openslr,denoised,এমন সংসদ সদস্য,3.348,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,...,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য
49149,/kaggle/input/denoiser-for-benchmark/denoised/...,7f87c122cb,ope

In [8]:
df=df[df.audio=='noisy']

In [9]:
n = 150
error_sen,actual_sen = df['wav2vec2'].to_list(),df['sentence'].to_list()

In [10]:
len(error_sen)

20713

In [11]:
chunked_list = list()
chunk_size = 1024
for i in range(0,len(error_sen),chunk_size):
    chunked_list.append(error_sen[i:i+chunk_size])

In [12]:
#chunked_list[0]

In [13]:
inference = []
for batch in chunked_list:
    batch = tokenizer(batch,truncation=True,padding=True, return_tensors="pt").to("cuda")
    translated = model.generate(**batch)
    inference += tokenizer.batch_decode(translated,skip_special_tokens=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
len(inference)

20713

In [15]:
df['wav2vec2_t5'] = inference

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df.sample(5)

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym,wav2vec2_t5
6676,/kaggle/input/denoiser-for-benchmark/raw/train...,31693320,train,noisy,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এলয় ...,5.904,-38.593491,শিল্পের মধ্যে অট মোবাইল বহনকারী ট্রেনএবং এলয় শ...,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এল শ...,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এলয় ...,...,40,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,শিল্পের মধ্যে অট মোবাইল বহনকারী ট্রেনিং এলয় শি...,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এল শ...,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এলয় ...,শিল্পের মধ্যে অটোমোবাইল বহনকারী ট্রেন এবং এলয় ...,শিল্পের মধ্যে অট মোবাইল বহনকারী ট্রেন এবং এলয়...
5440,/kaggle/input/denoiser-for-benchmark/raw/train...,31659297,train,noisy,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,7.092,-29.369987,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,...,0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত তি...,একাত্তরে বাংলাদেশ স্বাধীন হওয়ার পর প্রকাশিত ত...
31373,/kaggle/input/denoiser-for-benchmark/raw/slr53...,4af4599b68,openslr,noisy,দুর্বল লাইন,2.800,-26.353523,দুর্ব লাইন,দুর্বল লাইন,দুর্বল লাইন,...,50,"['C', 'C']",0,"['C', 'C']",0,দুর্বল লাইন,দুর্বল লাইন,দুর্বল লাইন,দুর্বল লাইন,দুর্বহ লাইন
32258,/kaggle/input/denoiser-for-benchmark/raw/slr53...,f10ee73370,openslr,noisy,তার জন্ম বরিশালে,4.100,-47.156773,তার জন্ম বরিশালে,তার জন্ম বরিশালে,তার জন্ম বরিশালে,...,0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,তার জন্ম বরিশালে,তার জন্ম বরিশালে,তার জন্ম বরিশালে,তার জন্ম বরিশালে,তার জন্ম বরিশালের
34537,/kaggle/input/denoiser-for-benchmark/raw/slr53...,1055ace456,openslr,noisy,উইকেটে রানেই,3.300,-23.988924,সাত উইকেট একশ ত্াতরৃসা নেই,সাত উইকেট এক তাতে নেই,সাত উইকেট এক তাতেই,...,100,"['I', 'I', 'S', 'S']",100,"['I', 'I', 'I', 'S', 'S']",100,সাত উইকেট একশ ত্াতরৃসা নেই,সাত উইকেট এক তাতে নেই,সাত উইকেট এক তাতেই,সাত উইকেট এক তাতে নেই,সাত উইকেট একশ রান নেই


In [17]:
df.to_parquet('benchmark_2.parquet',index=False)